In [1]:
import config as c 
from src.data import CombiDataset,make_loader,downsampling
from src.model import Model 
from src.freia_funcs import * 
from src.utils import * 
from tqdm import tqdm 
import tifffile  
import os 
import pandas as pd 
import torch.nn.functional as F 



my_experiment


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Eval 

In [2]:
from eval import eval 
score = {} 
all_classes = [d for d in os.listdir(c.dataset_dir) if os.path.isdir(os.path.join(c.dataset_dir, d))]

dataset_dir = c.dataset_dir
mode = c.data_mode
for class_name in all_classes:
#for class_name in ['grid']:
    print(f'\n Class : {class_name}')
    save_dir = os.path.join('./saved_models',dataset_dir.split('/')[-1],class_name)


    testset = CombiDataset(dataset_dir,class_name,mode,'test',device=c.device)
    testloader = make_loader(testset)

    teacher = torch.load(f'{save_dir}/teacher_best.pt')
    student = torch.load(f'{save_dir}/student_best.pt')

    mean_img_auc,max_img_auc = eval(teacher,student,testloader)
    score[class_name] = {'max' : max_img_auc,'mean' : mean_img_auc}
    
    


 Class : foam
Loaded pretrained weights for efficientnet-b5

 Class : tire
Loaded pretrained weights for efficientnet-b5

 Class : peach
Loaded pretrained weights for efficientnet-b5

 Class : cable_gland
Loaded pretrained weights for efficientnet-b5

 Class : carrot
Loaded pretrained weights for efficientnet-b5

 Class : rope
Loaded pretrained weights for efficientnet-b5

 Class : potato
Loaded pretrained weights for efficientnet-b5

 Class : cookie
Loaded pretrained weights for efficientnet-b5

 Class : bagel
Loaded pretrained weights for efficientnet-b5

 Class : dowel
Loaded pretrained weights for efficientnet-b5


In [5]:
mean_value = [] 
max_value = [] 
for i in score.keys():
    mean_value.append(score[i]['mean'])
    max_value.append(score[i]['max'])

df = pd.DataFrame([mean_value,max_value]).T
df.columns = ['mean','max']
df.index = score.keys()
df.loc['AVerage'] = np.mean(df).values
df.to_csv('./result.csv')

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [8]:
pwd

'/Volume/VAD/AST_3d'